In [25]:
# import packages
import numpy as np
import pandas as pd

In [26]:
# set workbook and sheet name
wb_name = 'ADEPT(2023-01-17).xlsm'
sh_name = 'Fluid-LUT'

#tags = ('depth', 'depth_ft', 'P_psig', 'T_F', 'yAsp_L1', 'yAsp_L2', beta_V, )

# get tags
df_tags = pd.read_excel(wb_name, sheet_name=sh_name, nrows=2)

# get numerical data
df = pd.read_excel(wb_name, sheet_name=sh_name, header=2)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 37 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   -             200 non-null    float64
 1   ft            200 non-null    float64
 2   psig          200 non-null    float64
 3   °F            200 non-null    float64
 4   g/cc          200 non-null    int64  
 5   cP            200 non-null    int64  
 6   Unnamed: 6    0 non-null      float64
 7   Unnamed: 7    0 non-null      float64
 8   gAsp_L1/gAsp  200 non-null    float64
 9   wtfrac        200 non-null    float64
 10  Unnamed: 10   200 non-null    float64
 11  wt%           200 non-null    int64  
 12  Unnamed: 12   200 non-null    float64
 13  Unnamed: 13   200 non-null    float64
 14  g/cc.1        200 non-null    int64  
 15  Unnamed: 15   200 non-null    float64
 16  Unnamed: 16   200 non-null    float64
 17  Unnamed: 17   200 non-null    float64
 18  MPa½          200 non-null    

In [27]:
data = df.to_numpy()

In [28]:
# unit conversions
def convert_TF_to_TK(TF):
    return (5/9)*(TF-32)+273.15

def convert_Ppsi_to_Pbar(Ppsi):
    return Ppsi * 0.0689475729

def convert_ft_to_m(zft):
    return zft * 0.3048

In [29]:
# convert to standard units
nZ, nProp = data.shape
data[:,1] = convert_ft_to_m(data[:,1])      #depth
data[:,2] = convert_Ppsi_to_Pbar(data[:,2]) #pressure
data[:,3] = convert_TF_to_TK(data[:,3])     #temperature
data[:,11:13] = data[:,11:13] / 100         #phase frac
data[:,14:17] = data[:,14:17] * 1000        #density       
data[:,18:22] = data[:,18:22] / 1000        #viscosity

In [30]:
depth=data[:,1], 
P=data[:,2], 
T=data[:,3],
Ceq=data[:,8],
yAsp_V=np.zeros(nZ),
yAsp_L1=data[:,9],
yAsp_L2=data[:,10],
wtFrac_V=data[:,11],
wtFrac_L1=data[:,12],
wtFrac_L2=data[:,13],
dens_V=data[:,14],
dens_L1=data[:,15],
dens_L2=data[:,16],
dens_Asp=data[:,17],
SP_V=data[:,18],
SP_L1=data[:,19],
SP_L2=data[:,20],
SP_Asp=data[:,21],
visco_V=data[:,22],
visco_L1=data[:,23],
visco_L2=data[:,24],
visco_Asp=data[:,25]